In [5]:
%history -g

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
from controller import Supervisor
from math import pi

In [7]:
supervisor = Supervisor()

Error: only one Robot instance can be created per controller process.


In [8]:
timestep = int(supervisor.getBasicTimeStep())

In [9]:
joints = []
joint_sensors = []
num_devices = supervisor.getNumberOfDevices()
for i in range(num_devices):
    device = supervisor.getDeviceByIndex(i)
    if not 'sensor' in device.getName():
        joints.append(device)
    else:
        joint_sensors.append(device)
        device.enable(timestep)

# Print the device names
for i, joint in enumerate(joints):
    print(joint)
    print(joint_sensors[i])
    joint.setPosition(float('inf'))
    joint.setVelocity(0)
    joint.setControlPID(1, 0, 0)


AttributeError: 'Supervisor' object has no attribute 'devices'

In [6]:
supervisor.step(timestep)

0

In [7]:
def get_joint_angles():
    angles = [joint_sensor.getValue() for joint_sensor in joint_sensors]
    angles[0] -= pi
    angles[1] += pi/2
    angles[3] += pi/2
    return angles

In [8]:
get_joint_angles()

In [9]:
import numpy as np

In [10]:
def move_to_config(target, duration=5):
    t0 = supervisor.getTime()
    v0 = np.zeros(6)
    vf = np.zeros(6)
    q0 = get_joint_angles()
    qf = np.array(target)
    a0 = np.zeros(6)
    af = np.zeros(6)
    tf = t0 + duration
    A = np.array([[1, t0, t0**2, t0**3, t0**4, t0**5],
                  [0, 1, 2*t0, 3*t0**2, 4*t0**3, 5*t0**4],
                  [0, 0, 2, 6*t0, 12*t0**2, 20*t0**3],
                  [1, tf, tf**2, tf**3, tf**4, tf**5],
                  [0, 1, 2*tf, 3*tf**2, 4*tf**3, 5*tf**4],
                  [0, 0, 2, 6*tf, 12*tf**2, 20*tf**3]])
    b = np.array([q0, v0, a0, qf, vf, af])
    x = [np.linalg.solve(A, b[:, i]) for i in range(6)]
    time0 = supervisor.getTime()
    iterations = 0
    end_effector_vel = []
    vel_jacob = [[], [], [], [], [], []]
    pos = [[], [], [], [], [], []]
    vel = [[], [], [], [], [], []]
    acc = [[], [], [], [], [], []]
    jerk = [[], [], [], [], [], []]
    time_arr = [[], [], [], [], [], []]
    while supervisor.getTime() <= tf:
        t = supervisor.getTime()
        for joint in joints:
            joint.setVelocity(x[joints.index(joint)][1] +
                              2*x[joints.index(joint)][2]*t +
                              3*x[joints.index(joint)][3]*t**2 +
                              4*x[joints.index(joint)][4]*t**3 +
                              5*x[joints.index(joint)][5]*t**4)


            idx = joints.index(joint)
            p = x[idx][0] + x[idx][1]*t + x[idx][2]*t**2 + x[idx][3]*t**3 + x[idx][4]*t**4 + x[idx][5]*t**5
            v = x[idx][1] + 2*x[idx][2]*t + 3*x[idx][3]*t**2 + 4*x[idx][4]*t**3 + 5*x[idx][5]*t**4
            a = 2*x[idx][2] + 6*x[idx][3]*t + 12*x[idx][4]*t**2 + 20*x[idx][5]*t**3
            j = 6*x[idx][3] + 24*x[idx][4]*t + 60*x[idx][5]*t**2
            time_arr[idx].append(t-time0)
            pos[idx].append(p)
            vel[idx].append(v)
            acc[idx].append(a)
            jerk[idx].append(j)
            v = x[idx][1] + 2*x[idx][2]*t + 3*x[idx][3]*t**2 + 4*x[idx][4]*t**3 + 5*x[idx][5]*t**4
            vel_jacob[idx].append(v)
        supervisor.step(timestep)
        iterations += 1
    for joint in joints:
        joint.setVelocity(0)
    timef = supervisor.getTime()
    error = np.abs(np.array(target) - get_joint_angles())*180/np.pi
    print('Iterações totais: ', iterations)
    print(timef-time0)
    print(np.max(error))
    print(np.mean(error))

In [11]:
move_to_config([.5*pi, .05*pi, .45*pi, 0, -.5*pi, .5*pi])

In [17]:
move_to_config([-.5*pi, -.05*pi, -.45*pi, 0, .5*pi, .5*pi])

In [14]:
move_to_config([pi/2, 0, 0, 0, 0, 0])

Iterações totais:  157
5.024000000000001
0.1356020285598345
0.032497642381255895


In [15]:
move_to_config([.1*pi, .18*pi, .57*pi, 0.25*pi, 0.2, pi/2])

Iterações totais:  157
5.023999999999999
0.061164707273728684
0.020201868583939905


In [16]:
move_to_config([-.3*pi, -.04*pi, -.8*pi, -0.17*pi, -0.5, pi/2])

Iterações totais:  157
5.023999999999999
0.008124500615705195
0.002446577905664432


In [19]:
move_to_config([0.2, .04*pi, .8*pi, 0.17*pi, 0.3, pi/2])

Iterações totais:  157
5.023999999999997
0.0009451299559592636
0.0005644981149434855
